# Predicting TV Show Success

## Notes
- Shonda Rhimes predictive superpower

- The problem statement
    - Find out which of the IVs/combination of IVs contributes the most to a shows success
    - Using this information after the pilot can you determine whether or not a show will be sucessful 
    - Using this information before the pilot can you determine whether or not a show will be sucessful 

- Possible independent variables:
    - producers/creators
    - network
    - viewer ratings
    - genre
    - air times (frequency and time of day)
    - actors
    - plot key words
    - certificate
    - runtime
    
- Operational definition of TV show success?

In [131]:
TITLE=[]
GENRE=[]
CREATED_BY=[]
STARRING=[]
NETWORK=[]
RUNNING_TIME=[]
NUM_SEASONS=[]
NUM_EPISODES=[]

TABLE_COLUMNS = ['title', 'genre', 'created_by', 'starring', 'network', 'running_time', 'num_seasons', 'num_episodes']

DATABASE = {'title':TITLE, 'genre':GENRE, 'created_by':CREATED_BY, 'starring':STARRING, 'network':NETWORK, 'running_time':RUNNING_TIME, 'num_seasons':NUM_SEASONS, 'num_episodes':NUM_EPISODES}

DATA_DIR = './data'
SYM_TABLE_NAME = 'sym_shows.csv'
NUM_TABLE_NAME = 'num_shows.csv'
A_TABLE_NAME = 'a_shows.csv'
B_TABLE_NAME = 'b_shows.csv'
C_TABLE_NAME = 'c_shows.csv'
D_TABLE_NAME = 'd_shows.csv'
E_TABLE_NAME = 'e_shows.csv'
F_TABLE_NAME = 'f_shows.csv'
G_TABLE_NAME = 'g_shows.csv'
H_TABLE_NAME = 'h_shows.csv'
I_TABLE_NAME = 'i_shows.csv'
J_TABLE_NAME = 'j_shows.csv'
K_TABLE_NAME = 'k_shows.csv'
L_TABLE_NAME = 'l_shows.csv'
M_TABLE_NAME = 'm_shows.csv'
N_TABLE_NAME = 'n_shows.csv'
O_TABLE_NAME = 'o_shows.csv'
P_TABLE_NAME = 'p_shows.csv'
Q_TABLE_NAME = 'q_shows.csv'
R_TABLE_NAME = 'r_shows.csv'
S_TABLE_NAME = 's_shows.csv'
T_TABLE_NAME = 't_shows.csv'
U_TABLE_NAME = 'u_shows.csv'
V_TABLE_NAME = 'v_shows.csv'
W_TABLE_NAME = 'w_shows.csv'
X_TABLE_NAME = 'x_shows.csv'
Y_TABLE_NAME = 'y_shows.csv'
Z_TABLE_NAME = 'z_shows.csv'

In [132]:
#Collect a list of Wikipedia links to all American TV shows

from bs4 import BeautifulSoup as soup
import pandas as pd
import re
from urllib import request
 
def soupTheLink(url):
    html = request.urlopen(url).read().decode('utf8')
    return soup(html, 'lxml')

def collectLinks(url):
    souped = soupTheLink(url)
    
    showLinks = []
    
    for listItem in souped.find_all('li'):
        link = listItem.find('a', href = True)
        try:
            fullLink = 'https://en.wikipedia.org' + link['href']
            showLinks.append(fullLink)
        except TypeError:
            pass
    showLinks = showLinks[28:2282]
    return showLinks

def compileData(listOfShowLinks):
    print("number of shows = ", len(listOfShowLinks))
    count = 0
    for url in listOfShowLinks[923:1054]:
        count += 1
        print('show number ', count)
        souped = soupTheLink(url)

        table_num = 0
        for table in souped.find_all(class_='infobox'):
            table_num += 1
            if table_num == 1:
                has_genre = 0
                has_created_by = 0
                has_starring = 0
                has_network = 0
                has_release = 0
                has_running_time = 0
                has_num_seasons = 0
                has_num_episodes = 0

                for row in table.find_all('tr'):

                    for category in row.find_all('th'):
                        feature = category.contents

                        if feature == ['Genre']:
                            has_genre = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                for genre in info.find_all('a'):
                                    rowContent.extend(genre.contents)
                            print('Genre: ', rowContent)

                            if rowContent == []:
                                for info in row.find_all('td'):
                                    rowContent.extend(info.contents)
                                print('Genre: ', rowContent)

                            DATABASE['genre'].append(rowContent)

                        elif feature == ['Created by'] or feature == ['Written by'] and has_created_by == 0:
                            has_created_by = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                creators = []

                                for creator in info.find_all('a'):
                                    name = str(creator.contents)
                                    if name[2] != '[':
                                        creators.extend(creator.contents)
                                rowContent.extend(creators)
                            print('Created by: ', rowContent)

                            if rowContent == []:
                                for info in row.find_all('td'):
                                    rowContent.extend(info.contents)
                                rowContent = rowContent[0]
                                print('Created by: ', rowContent)

                            DATABASE['created_by'].append(rowContent)

                        elif feature == ['Starring']:
                            has_starring = 1

                            rowContent = []

                            for info in row.find_all('td'):

                                for stars in info.find_all('a'):
                                    rowContent.extend(stars.contents)

                            print('Starring: ', rowContent)

                            if rowContent == []:
                                for info in row.find_all('td'):
                                    rowContent.extend(info.contents)
                                rowContent = rowContent[0]
                                print('Starring: ', rowContent)

                            DATABASE['starring'].append(rowContent)

                        elif feature == ['Original network']:
                            has_network = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                for network in info.find_all('a'):
                                    rowContent.extend(network.contents)
                            print('Network: ', rowContent)

                            DATABASE['network'].append(rowContent)

                        elif feature == ['Running time']:
                            has_running_time = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                rowContent = info.contents
                                print('Running time: ', rowContent)

                                rowContent = rowContent[0]
                                rowContent = rowContent[0:2]

                                DATABASE['running_time'].append(rowContent)

                        elif feature[-1] == ' of seasons':
                            has_num_seasons = 1

                            for info in row.find_all('td'):
                                rowContent = info.contents[0]
                                print('Seasons: ', rowContent)

                                DATABASE['num_seasons'].append(rowContent)

                        elif feature[-1] == ' of episodes':
                            has_num_episodes = 1

                            for info in row.find_all('td'):
                                rowContent = info.contents[0]
                                print('Episodes:', rowContent)
                                DATABASE['num_episodes'].append(rowContent)


                        else:
                            continue
                    
                if has_genre == 0:
                    print('Genre: None')
                    DATABASE['genre'].append("None")
                if has_created_by == 0:
                    print('Created by: None')
                    DATABASE['created_by'].append("None")
                if has_starring == 0:
                    print('Starring: None')
                    DATABASE['starring'].append("None")
                if has_network == 0:
                    print('Network: None')
                    DATABASE['network'].append("None")
                if has_running_time == 0:
                    print('Running time: None')
                    DATABASE['running_time'].append("None")
                if has_num_seasons == 0:
                    print('Seasons: None')
                    DATABASE['num_seasons'].append("None")
                if has_num_episodes == 0:
                    print('Episodes: None')
                    DATABASE['num_episodes'].append("None")
        if table_num > 0:
            titleCount = 0
            for title in souped.find_all(class_='firstHeading'):
                for content in title.find_all('i'):
                    titleCount = 1
                    showTitle = content.contents
                    print(showTitle)
                    DATABASE['title'].append(showTitle)
                if titleCount == 0:
                    showTitle = title.contents
                    print(showTitle)
                    DATABASE['title'].append(showTitle)

In [3]:
mainUrl = 'https://en.wikipedia.org/wiki/List_of_American_television_series'

In [4]:
links = collectLinks(mainUrl)

In [133]:
compileData(links)

number of shows =  2254
show number  1
Genre:  ['Western']
Running time:  ['30 minutes']
Starring:  ['William Conrad', 'Parley Baer', 'Howard McNear', 'Georgia Ellis']
Created by: None
Network: None
Seasons: None
Episodes: None
['Gunsmoke']
show number  2
Created by:  ['Sid and Marty Krofft']
Starring:  ['Jack Wild', 'Billie Hayes']
Episodes: 17
Running time:  ['0:25 (per episode)']
Network:  ['NBC']
Genre: None
Seasons: None
['H.R. Pufnstuf']
show number  3
Created by:  ['David Koepp']
Starring:  ['David Morse', 'George Dzundza', 'Andre Braugher', 'Jacqueline Torres']
Seasons:  2
Episodes: 40
Running time:  ['60 minutes']
Network:  ['CBS', 'M6', 'Network Ten', 'ITV3']
Genre: None
['Hack']
show number  4
Created by:  ['Jeffrey Klarik']
Starring:  ['Rachel True', 'Essence Atkins', 'Chico Benymon', 'Valarie Pettiford', 'Alec Mapa', 'Telma Hopkins']
Seasons:  4
Episodes: 91
Running time:  ['23 minutes']
Network:  ['UPN']
Genre: None
['Half & Half']
show number  5
Created by:  ['Oscar Nune

IndexError: list index out of range

In [123]:
print(DATABASE)

{'num_episodes': ['17', '40', '91', '10 ', '250+ ', '20 ', '16 (2 unaired)', '104 ', '101 ', '26', '10 (5 unaired) ', '98 ', '39 ', 'None', '34 ', '255 ', '57 ', '22', '13 (9 unaired)', '8 (2 unaired) ', '128 ', '2159', 'None', '67', '46 ', '10', '30', '13 ', '72', '6', '44', '15', '34 ', '\n', '39 ', '11 ', '225 ', '78 ', '8', '279 ', '143 ', '134 ', '30 ', '154 ', '130 ', '6 (4 unaired)', '114 ', '14', '9', '54 (1 unaired)', 'None', 'None', '13 (1 unaired)', '7 (2 unaired)', '13', '22 ', '38 (', '234 ', '13 (4 unaired) ', '65 ', '111', '52', '14', '144 ', '72 ', 'None', '77 ', '100 (185 segments) ', '65 ', '18 (5 unaired)', '8 ', '98 ', '19 ', '12 ', 'None', '119 (', '156', '111 ', '146 ', '102 ', '29', '6', '85', '110 ', '43', '168 (Pilot episode-B/W; 167-color) ', '18 (8 unaired) (Fox)', '80 ', '3,536 (NBC daytime)', '204 ', '52 ', '26', '42 (', '60 ', '13 (2 unaired)', '122 ', '66 ', '30', '39 ', '42', 'None', '43', '73 ', '35', '128 ', '13', '13', '114', '177 ', '57', '190 (+ 90-

In [124]:
print(len(TITLE))
print(len(GENRE))
print(len(CREATED_BY))
print(len(STARRING))
print(len(NETWORK))
print(len(RUNNING_TIME))
print(len(NUM_SEASONS))
print(len(NUM_EPISODES))

120
120
120
120
120
120
120
120


In [27]:
df = pd.DataFrame(DATABASE, columns = TABLE_COLUMNS)

ValueError: arrays must all be same length

In [21]:
df.head()

,title,genre,created_by,starring,network,running_time,num_seasons,num_episodes
0,$#*! My Dad Says,[Sitcom],"[David Kohan, Max Mutchnick, Justin Halpern]","[William Shatner, Jonathan Sadowski, Will Sass...",[CBS],21,1,18
1,"The $64,000 Question",[Game show],[Joseph Nathan Kane],None,[CBS],22,4,None
2,"The $64,000 Question",[Game show],[Joseph Nathan Kane],None,[CBS],22,4,None
3,1 vs. 100,[Game show],None,None,"[NBC, GSN]",41,3,68
4,1st & Ten,None,Carl Kleinschmitt,"[Delta Burke, O. J. Simpson, Shannon Tweed, Jo...",[HBO],25,6,80


In [10]:
dogss = []
print(dogss == [])

dogss = ['pups']

print(dogss == [])

True
False
